In [1]:
# # https://www.kaggle.com/shivamkushwaha/bbc-full-text-document-classification
# !wget -nc https://lazyprogrammer.me/course_files/nlp/bbc_text_cls.csv


In [2]:
import numpy as np
import pandas as pd

import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, GlobalAvgPool1D
from tensorflow.keras.layers import Conv1D , MaxPooling1D,Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.losses import  SparseCategoricalCrossentropy

In [3]:
df = pd.read_csv('bbc_text_cls.csv')
df.head()

,text,labels
0,Ad sales boost Time Warner profit\n\nQuarterly...,business
1,Dollar gains on Greenspan speech\n\nThe dollar...,business
2,Yukos unit buyer faces loan claim\n\nThe owner...,business
3,High fuel prices hit BA's profits\n\nBritish A...,business
4,Pernod takeover talk lifts Domecq\n\nShares in...,business


In [4]:
df['targets'] = df['labels'].astype('category').cat.codes

In [5]:
k = df['targets'].max() + 1
k

5

In [6]:
df_train,df_test = train_test_split(df, test_size=0.3)

In [7]:
MAX_VOCAB_SIZE = 2000
tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE)
tokenizer.fit_on_texts(df_train['text'])
sequences_train = tokenizer.texts_to_sequences(df_train['text'])
sequences_test = tokenizer.texts_to_sequences(df_test['text'])

In [8]:
word2idx = tokenizer.word_index
V = len(word2idx)
print('Found %s unique tokens.' % V)

Found 27608 unique tokens.


In [9]:
data_train  = pad_sequences(sequences_train)
print('Shape of data train tensor:', data_train.shape)

Shape of data train tensor: (1557, 2852)


In [10]:
T = data_train.shape[1]

In [11]:
data_test  = pad_sequences(sequences_test,maxlen=T)
print('Shape of data test tensor:', data_test.shape)

Shape of data test tensor: (668, 2852)


In [12]:
D = 50

i = Input(shape=(T,))
x = Embedding(V + 1, D)(i)
x = Conv1D(32,3,activation='relu')(x)
x = GlobalAvgPool1D()(x)
x = Dense(k)(x)

model = Model(i, x)

In [13]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 2852)]            0         
                                                                 
 embedding (Embedding)       (None, 2852, 50)          1380450   
                                                                 
 conv1d (Conv1D)             (None, 2850, 32)          4832      
                                                                 
 global_average_pooling1d (  (None, 32)                0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 5)                 165       
                                                                 
Total params: 1385447 (5.29 MB)
Trainable params: 1385447 (5.29 MB)
Non-trainable params: 0 (0.00 Byte)
_______________________

In [14]:
model.compile(loss=SparseCategoricalCrossentropy(from_logits=True),
              optimizer='adam',metrics =['accuracy'])

In [15]:
print(data_train.shape)
print(df_train['targets'].shape)

(1557, 2852)
(1557,)


In [17]:
print(data_test.shape)
print(df_test['targets'].shape)

(668, 2852)
(668,)


In [18]:
print('Training model.....')

r = model.fit(data_train, df_train['targets'],epochs=50,
              validation_data=(data_test,df_test['targets']))


Training model.....
Epoch 1/50
49/49 [==============================] - 9s 157ms/step - loss: 1.6025 - accuracy: 0.2222 - val_loss: 1.6019 - val_accuracy: 0.2081
Epoch 2/50
49/49 [==============================] - 6s 125ms/step - loss: 1.5849 - accuracy: 0.2364 - val_loss: 1.5784 - val_accuracy: 0.2126
Epoch 3/50
49/49 [==============================] - 7s 150ms/step - loss: 1.5395 - accuracy: 0.3629 - val_loss: 1.5060 - val_accuracy: 0.5374
Epoch 4/50
49/49 [==============================] - 7s 146ms/step - loss: 1.4514 - accuracy: 0.5106 - val_loss: 1.3985 - val_accuracy: 0.4626
Epoch 5/50
49/49 [==============================] - 8s 160ms/step - loss: 1.3286 - accuracy: 0.6204 - val_loss: 1.2731 - val_accuracy: 0.6542
Epoch 6/50
49/49 [==============================] - 9s 178ms/step - loss: 1.1955 - accuracy: 0.6943 - val_loss: 1.1428 - val_accuracy: 0.6796
Epoch 7/50
49/49 [==============================] - 7s 128ms/step - loss: 1.0686 - accuracy: 0.7219 - val_loss: 1.0278 - val_acc